## _*Using Qiskit Aqua for clique problems*_

This Qiskit Aqua Optimization notebook demonstrates how to use the VQE quantum algorithm to compute the clique of a given graph. 

The problem is defined as follows. A clique in a graph $G$ is a complete subgraph of $G$. That is, it is a subset $K$ of the vertices such that every two vertices in $K$ are the two endpoints of an edge in $G$. A maximal clique is a clique to which no more vertices can be added. A maximum clique is a clique that includes the largest possible number of vertices. 

We will go through three examples to show (1) how to run the optimization in the non-programming way, (2) how to run the optimization in the programming way, (3) how to run the optimization with the VQE.
We will omit the details for the support of CPLEX, which are explained in other notebooks such as maxcut.

Note that the solution may not be unique.

###  The problem and a brute-force method.

In [1]:
import numpy as np

from qiskit import Aer

from qiskit_aqua import run_algorithm
from qiskit_aqua.input import EnergyInput
from qiskit_aqua.translators.ising import clique
from qiskit_aqua.algorithms import ExactEigensolver

first, let us have a look at the graph, which is in the adjacent matrix form.

In [2]:
K = 3  # K means the size of the clique
np.random.seed(100)
num_nodes = 5
w = clique.random_graph(num_nodes, edge_prob=0.8, weight_range=10)
print(w) 

[[ 0.  4.  5.  3. -5.]
 [ 4.  0.  7.  0.  6.]
 [ 5.  7.  0. -4.  0.]
 [ 3.  0. -4.  0.  8.]
 [-5.  6.  0.  8.  0.]]


Let us try a brute-force method. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a vertex is either 0 (meaning the vertex is not in the clique) or 1 (meaning the vertex is in the clique). We print the binary assignment that satisfies the definition of the clique (Note the size is specified as K).

In [3]:
def brute_force():
    # brute-force way: try every possible assignment!
    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]

    L = num_nodes  # length of the bitstring that represents the assignment
    max = 2**L
    has_sol = False
    for i in range(max):
        cur = bitfield(i, L)
        cur_v = clique.satisfy_or_not(np.array(cur), w, K)
        if cur_v:
            has_sol = True
            break
    return has_sol, cur

has_sol, sol = brute_force()
if has_sol:
    print("solution is ", sol)
else:
    print("no solution found for K=", K)

solution is  [1, 0, 0, 1, 1]


### Part I: run the optimization in the non-programming way

In [4]:
qubit_op, offset = clique.get_clique_qubitops(w, K)
algo_input = EnergyInput(qubit_op)
params = {
    'problem': {'name': 'ising'},
    'algorithm': {'name': 'ExactEigensolver'}
}
result = run_algorithm(params, algo_input)
x = clique.sample_most_likely(len(w), result['eigvecs'][0])
ising_sol = clique.get_graph_solution(x)
if clique.satisfy_or_not(ising_sol, w, K):
    print("solution is", ising_sol)
else:
    print("no solution found for K=", K)

solution is [1. 0. 1. 1. 0.]


### Part II: run the optimization in the programming way

In [5]:

algo = ExactEigensolver(algo_input.qubit_op, k=1, aux_operators=[])
result = algo.run()
x = clique.sample_most_likely(len(w), result['eigvecs'][0])
ising_sol = clique.get_graph_solution(x)
if clique.satisfy_or_not(ising_sol, w, K):
    print("solution is", ising_sol)
else:
    print("no solution found for K=", K)     

solution is [1. 0. 1. 1. 0.]


### Part III: run the optimization with the VQE

In [6]:
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'COBYLA'
}

var_form_cfg = {
    'name': 'RY',
    'depth': 5,
    'entanglement': 'linear'
}

params = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg
}
backend = Aer.get_backend('statevector_simulator')
result = run_algorithm(params, algo_input, backend=backend)
x = clique.sample_most_likely(len(w), result['eigvecs'][0])
ising_sol = clique.get_graph_solution(x)

if clique.satisfy_or_not(ising_sol, w, K):
    print("solution is", ising_sol)
else:
    print("no solution found for K=", K)

solution is [1. 0. 1. 1. 0.]
